In [1]:
from datasets import DataAugmentationForVideoMAE
from kinetics import VideoMAE2VID
from types import SimpleNamespace as Namespace
from torch.utils.data import DataLoader 
import torch
from timm.models import create_model
import os.path as osp
import copy
from einops import rearrange
from utils import  clone_decoder_weights
import run_videomae_vis_v2  as vis_utils 
# unnormalize_frames, save_video, reconstruct_video_from_patches
import importlib
from datasets import build_pretraining_dataset_multi_decoder
# importlib.reload(VideoMAE2VID)


In [2]:
def get_model(args):
    print(f"Creating model: {args.model}")
    model = create_model(
        args.model,
        pretrained=False,
        drop_path_rate=args.drop_path,
        drop_block_rate=None,
        decoder_depth=args.decoder_depth,
        clone_decoder = args.clone_decoder
    )

    return model

In [3]:
# def clone_decoder_weights(n, original_dict):

#         # New dictionary to store updated keys and values
#     new_dict = {}

#         # Iterate over original dictionary
#         # Iterate over original dictionary
#     for key, value in original_dict.items():
#         # Check if the key contains the word 'decoder.'
#         if key.startswith('decoder.'):
            
#             # Generate new keys based on the pattern
#             new_keys = [key.replace('decoder.', f'decoder.decoders.{i}.') for i in range(n)]
            
#             # Update new dictionary with new keys and cloned values
#             for new_key in new_keys:
#                 new_dict[new_key] = value.clone()
#         else:
#             # If key doesn't contain 'decoder.', simply copy the original key-value pair
#             new_dict[key] = value
#     return new_dict

In [4]:
# def build_pretraining_dataset(args):
#     transform = DataAugmentationForVideoMAE(args)
#     dataset = VideoMAE2VID(
#         root=args.root,
#         setting=args.data_path,
#         is_color=True,
#         modality='rgb',
#         new_length=args.num_frames,
#         new_step=args.sampling_rate,
#         transform=transform,
#         temporal_jitter=False,
#         video_loader=True,
#         use_decord=True,
#         lazy_init=False,
#         features_cfg=args.features_cfg,

#         )
#     print("Data Aug = %s" % str(transform))
#     return dataset

In [5]:
# checkpoint_name = 'VideoMAE_ViT-B_checkpoint_Kinetics-400.pth'
# model_path = osp.join('/videos/pretrained/pretrained',checkpoint_name)
model_path = osp.join('/videos/pretrained/MPIIGroupInteraction/k400_finetune_videomae_pretrain_dual_patch16_224_frame_16x4_tube_mask_ratio_0.9_e100/checkpoint-99.pth')

model_name = 'pretrain_videomae_base_patch16_224_densepose_dual'
# model_name = 'pretrain_videomae_base_patch16_224'


In [7]:
features_cfg ={
    'videos': {'folder': 'clips_train', 'file_extention':'mp4', 'loss_weight':0.5},
    'densepose': {'folder': 'densepose_train', 'file_extention':'mp4', 'loss_weight':0.5},
}

In [8]:
args = Namespace()
args.root = '/videos/mpi_data/2Itzik/MPIIGroupInteraction/'
args.data_path = 'train_data_mpig.txt'
args.features_cfg = features_cfg
args.mask_type = 'tube' 
args.mask_ratio = 0.9 
args.decoder_depth = 4 
args.batch_size = 32 
args.num_frames = 16 
args.sampling_rate = 4 
args.input_size = 224
args.patch_size = 16
args.window_size = (args.num_frames // 2, args.input_size // args.patch_size, args.input_size // args.patch_size)
args.model_path=model_path
args.device='cuda:0'
args.imagenet_default_mean_and_std=True
args.model=model_name
args.drop_path=0.0
args.clone_decoder = False
args.feature_cfg = features_cfg



NameError: name 'model_path' is not defined

In [9]:
device = torch.device(args.device)

model = get_model(args)
patch_size = model.encoder.patch_embed.patch_size
print("Patch size = %s" % str(patch_size))
args.window_size = (args.num_frames // 2, args.input_size // patch_size[0], args.input_size // patch_size[1])
args.patch_size = patch_size

model.to(device)
checkpoint = torch.load(args.model_path, map_location='cpu')
n = model.decoder.num_decoders
state_dict = clone_decoder_weights(n, checkpoint['model'])
# state_dict = checkpoint['model']
model.load_state_dict(state_dict)
model.eval()


AttributeError: 'types.SimpleNamespace' object has no attribute 'device'

In [10]:
model.state_dict()['decoder.decoders.1.blocks.']

NameError: name 'model' is not defined

In [11]:
model.decoder.num_decoders

NameError: name 'model' is not defined

In [ ]:
# dataset = build_pretraining_dataset(args=args)
dataset = build_pretraining_dataset_multi_decoder(args=args)

In [ ]:
dataloader = DataLoader(dataset, batch_size=3, shuffle=False)


In [ ]:
# list(dataloader.batch_sampler)

In [ ]:
# iterator_dataset =iter(dataset)
# batch =  next(iterator_dataset)

In [ ]:
for i , (batch, mask) in enumerate(dataloader):
    print('new batch')
    for b in batch:
        print(b.shape)
    print(i)
    if i >= 1:
        break
    

In [ ]:
len(batch[0])

In [ ]:

ori_img = vis_utils.unnormalize_frames(img=batch[0])
# ori_img = vis_utils.unnormalize_frames(img=batch[1])
video_save_path = 'testing_this_deletelater.mp4'
vis_utils.save_video(ori_img[0], video_save_path,frame_id_list=None, prepend=None)

In [ ]:
model_input = batch[0].to(device)
mask_input = mask.flatten(1).to(torch.bool).to(device)
mask_input = torch.zeros_like(mask_input).to(torch.bool).to(device)
outputs = model(model_input, mask_input)

In [ ]:
torch.max(outputs[0])

In [ ]:
# img_squeeze = rearrange(batch[0], 'b c (t p0) (h p1) (w p2) -> b (t h w) (p0 p1 p2) c', p0=2, p1=patch_size[0], p2=patch_size[0])
# img_patch = rearrange(img_squeeze, 'b n p c -> b n (p c)')

# model_output = outputs[0]

# model_output = model_output.detach().cpu()

In [ ]:
print(torch.sum(outputs[0] == outputs[1]))
print(outputs[0].numel())

In [ ]:
# importlib.reload(vis_utils)
# 

In [ ]:
# bool_masked_pos = mask_input
bool_masked_pos = torch.zeros_like(mask_input)
frame_id_list = None
model_out = outputs[1]
imgs, rec_img, mask_2 = vis_utils.reconstruct_video_from_patches(ori_img.to(device), patch_size, bool_masked_pos, model_out, frame_id_list)

In [ ]:
# rec_img.shape
vis_utils.save_video(rec_img, video_save_path,frame_id_list=None, prepend=None)

In [ ]:
# dry test with run_pretrain
checkpoint_name = 'VideoMAE_ViT-B_checkpoint_Kinetics-400.pth'
model_path = osp.join('/videos/pretrained/pretrained',checkpoint_name)

model_name = 'pretrain_videomae_base_patch16_224_densepose_dual'

import argparse

# Create args Namespace with default values
args = argparse.Namespace(
    batch_size=64,
    epochs=800,
    save_ckpt_freq=50,
    decoder_depth=4,
    mask_type='tube',
    mask_ratio=0.75,
    input_size=224,
    drop_path=0.0,
    normlize_target=True,
    opt='adamw',
    opt_eps=1e-8,
    momentum=0.9,
    weight_decay=0.05,
    lr=1.5e-4,
    warmup_lr=1e-6,
    min_lr=1e-5,
    warmup_epochs=40,
    warmup_steps=-1,
    use_checkpoint=False,
    color_jitter=0.0,
    train_interpolation='bicubic',
    data_path='/path/to/list_kinetics-400',
    imagenet_default_mean_and_std=True,
    num_frames=16,
    sampling_rate=4,
    output_dir='',
    log_dir=None,
    device='cuda',
    seed=0,
    auto_resume=True,
    start_epoch=0,
    num_workers=10,
    pin_mem=True,
    world_size=1,
    local_rank=-1,
    dist_on_itp=False,
    dist_url='env://',
    features_cfg=None,
    clone_decoder=False,
    weight_decay_end=None,
    resume='',
    clip_grad=None,
)

# Override default values with values provided in the call from terminal
args.batch_size = 2
args.lr = 0.000003
args.save_ckpt_freq = 20
args.epochs = 500
args.log_dir = 'testing_new_model_debug'
args.output_dir = 'testing_new_model_debug'

args.root = '/videos/mpi_data/2Itzik/MPIIGroupInteraction/'
args.data_path = 'train_data_mpig.txt'
args.mask_type = 'tube' 
args.mask_ratio = 0.9 
args.decoder_depth = 4 
args.batch_size = 2 
args.num_frames = 16 
args.sampling_rate = 4 
args.input_size = 224
args.patch_size = 16
args.window_size = (args.num_frames // 2, args.input_size // args.patch_size, args.input_size // args.patch_size)
args.model_path=model_path
args.device='cuda:0'
args.imagenet_default_mean_and_std=True
args.model=model_name
args.drop_path=0.0
args.clone_decoder = False # True
args.feature_cfg = False





In [ ]:
args.feature_cfg

In [ ]:
import run_mae_pretraining_multi_decoder
importlib.reload(run_mae_pretraining_multi_decoder)

In [5]:
run_mae_pretraining_multi_decoder.main(args)

ERROR:tornado.general:SEND Error: Host unreachable


In [ ]:
model.decoder.num_features